# Issue Commands to Change Settings

https://github.com/stilldavid/gopro-utils

In [1]:
import os
import time
from collections import OrderedDict

import IPython
import ipywidgets

import data_io
from progress_bar import bar

import context

import gopro_helper as gopro

# Camera Current Settings

In [4]:
wid_stat = gopro.GoProStatus(interval=5)

In [26]:
wid_stat.stop()

# New Stuff

In [7]:
# # Photo feature IDs
# photo = gopro.namespace.Struct()
# video = gopro.namespace.Struct()

# photo.white = 22
# photo.color = 23
# photo.EV = 26
# photo.protune = 21
# photo.resolution = 17
# photo.RAW = 82
# photo.WDR = 77
# photo.ISO_min = 75
# photo.ISO_max = 24
# photo.shutter = 97

# video.resolution = 2
# video.FPS = 3
# video.FOV = 4
# video.low_light = 8
# video.stab = 78
# video.white = 11
# video.color = 12
# video.sharp = 14
# video.EV = 15
# video.shutter = 73
# video.ISO = 13
# video.ISO_mode = 74
# video.protune = 10

# feature_id = gopro.namespace.Struct()
# feature_id.photo = photo
# feature_id.video = video

In [142]:
# def feature_choices(mode, fid):
#     details = _mode_details(mode)

#     options = OrderedDict()
#     for item in details:
#         if item['id'] == fid:
#             name = item['display_name']
#             for entry in item['options']:
#                 options[entry['display_name']] = entry['value']
#             return name, options


def fid_mode(fid):
    """Determine mode to which specified feature ID belongs
    """
    mode = None
    for m in ['photo', 'video']:        
        for item in _mode_details(m):
            if item['id'] == fid:
                if not mode:
                    mode = m
                else:
                    raise ValueError('Found multiple entries for feature: {}'.format(fid))

    return mode



def feature_current_value(fid):
    """Return camera's current value for specified feature ID
    """
    content = gopro.get(gopro.api.url_status)
    settings = content['settings']
    
    try:
        value = settings[str(fid)]
    except KeyError:
        raise ValueError('Invalid feature ID {}'.format(fid))
        
    return value






def select_feature_widget(fid, value_init=0):
    mode = fid_mode(fid)
    
    name, options = feature_choices(mode, fid)
    
    wid = ipywidgets.Dropdown(value=value_init, options=options, description=name)
    wid.layout.width='200pt'

    return wid



def multi_feature_dropdown():
    
    mode = gopro.status.current_mode()

    widgets = []
    for feature_name, fid in feature_id[mode].items():
        value_init = feature_current_value(fid)
        wid_feature = select_feature_value(fid, value_init=value_init)
    
        def handle_feature_selection(event):
            value = event.new
            if value > -1:
                print('set {} {}'.format(fid, value))
#                set_feature_value(fid, value)
        

        wid_feature.observe(handle_feature_selection, names='value')
        
        widgets.append(wid_feature)
  
    box = ipywidgets.VBox(widgets)
    
    IPython.display.display(box)

In [145]:
def feature_value(fid):
    mode = gopro.status.current_mode()
    
    assert(mode == fid_mode(fid))
    
    value_init = feature_current_value(fid)
    wid_feature = select_feature_widget(fid, value_init=value_init)
    
    def handle_feature_selection(event):
        value = event.new
#         print('set {} {}'.format(fid, value))
        set_feature_value(fid, value)
        
    wid_feature.observe(handle_feature_selection, names='value')
  
    IPython.display.display(wid_feature)

In [169]:
feature_value(feature_id.video.resolution)

In [146]:
feature_value(feature_id.video.FPS)

In [147]:
feature_value(feature_id.video.ISO_mode)

In [170]:
feature_value(feature_id.video.ISO)

Internal Server Error
500


In [51]:
wid_stat = gopro.GoProStatus(interval=1)

In [41]:
wid_stat.stop()

In [171]:
gopro.commands.get_status()

{'settings': {'1': 0,
  '10': 1,
  '11': 6,
  '12': 0,
  '13': 2,
  '14': 2,
  '15': 4,
  '16': 0,
  '17': 10,
  '18': 4,
  '19': 0,
  '2': 7,
  '20': 0,
  '21': 1,
  '22': 1,
  '23': 0,
  '24': 1,
  '25': 2,
  '26': 4,
  '27': 0,
  '28': 0,
  '29': 1,
  '3': 5,
  '30': 1,
  '31': 0,
  '32': 4,
  '33': 0,
  '34': 0,
  '35': 0,
  '36': 0,
  '37': 4,
  '38': 0,
  '39': 4,
  '4': 0,
  '40': 0,
  '41': 13,
  '42': 8,
  '43': 0,
  '44': 8,
  '45': 8,
  '46': 0,
  '47': 0,
  '48': 0,
  '5': 1,
  '50': 0,
  '51': 1,
  '52': 0,
  '54': 0,
  '57': 0,
  '58': 1,
  '59': 0,
  '6': 1,
  '60': 8,
  '61': 1,
  '62': 2500000,
  '63': 7,
  '64': 4,
  '65': 0,
  '66': 0,
  '67': 0,
  '68': 0,
  '69': 1,
  '7': 1,
  '70': 0,
  '71': 0,
  '72': 1,
  '73': 0,
  '74': 0,
  '75': 2,
  '76': 3,
  '77': 0,
  '78': 0,
  '79': 1,
  '8': 1,
  '80': 0,
  '81': 0,
  '82': 0,
  '83': 1,
  '84': 0,
  '85': 0,
  '86': 0,
  '87': 40,
  '88': 51,
  '89': 12,
  '9': 0,
  '91': 2,
  '92': 12,
  '93': 0,
  '94': 0,
  '95'

In [167]:
gopro.status.fetch_camera_info(False)

Struct([('video',
         Struct([('Default Video Sub Mode', 'Video'),
                 ('Video Sub Mode', 'Video'),
                 ('Resolution', '1440'),
                 ('Frames Per Second', '60'),
                 ('Field of View', 'Wide'),
                 ('Interval', '1 Photo / 5 Seconds'),
                 ('Low Light', 'ON'),
                 ('Video Stabilization', 'OFF'),
                 ('Protune', 'ON'),
                 ('White Balance', '4800K'),
                 ('Color', 'GoPro'),
                 ('Sharpness', 'Low'),
                 ('EV Comp', '0.0'),
                 ('Shutter', 'Auto'),
                 ('ISO', '400'),
                 ('ISO Mode', 'Max'),
                 ('No Audio Track', 'OFF')])),
        ('setup',
         Struct([('Screensaver', '1 MIN'),
                 ('Auto-Rotation', 'Auto'),
                 ('Default Mode', 'Video'),
                 ('Current Flat Mode', 'Video'),
                 ('Quick Capture', 'OFF'),
                 ('

In [62]:
url = gopro.api.tpl_setting.format(feature=75, value=2)

url

'http://10.5.5.9/gp/gpControl/setting/75/2'

In [63]:
# gopro.api.get(gopro.api.url_mode_photo)
gopro.api.get(url)

{}

# Timing

In [39]:
gopro.commands.shutter_capture()

time_max = 5
interval = 0.05

time_start = time.time()
while time.time() - time_start < time_max:
    
    time_pre = time.time()
    stat = gopro.status.fetch_camera_info(pretty=False)    
    time_after = time.time()

    delta = time_after - time_pre
    time_sleep = interval - delta
    
    if stat['System']['system_busy'] > 0:
        if time_sleep > 0.001:
            time.sleep(time_sleep)
    else:
        break

print('{:.3f} sec'.format(time_after - time_start))

1.643 sec


# Basic Commands

In [19]:
gopro.commands.set_mode_photo()

In [16]:
gopro.commands.set_mode_video()

In [53]:
gopro.commands.shutter_capture()

In [52]:
gopro.commands.shutter_stop()

# File Transfer

In [51]:
gopro.commands.update_local_data(delete=True)